# Webscraping WKÖ Fimen A-Z 

# Import libraries 

In [ ]:
import pandas as pd
import requests

from bs4 import BeautifulSoup
import urllib.parse

# Defining extract function

In [ ]:
def extract_info(entry):
    
    firma_name = entry.find('a').get_text()

    firma_href = entry.find('a').get('href')
    # use tmp = urllib.parse.parse_qs(firma_href) to extract dictionary from url

    #tmp = urllib.parse.parse_qs(firma_href)
    #firma_id = tmp['/Web/DetailsKontakt.aspx?FirmaID']
    #firma_branche = tmp['Branche']
    #firma_branchenname = tmp['BranchenName']

    try:
        firma_str = entry.find('div', itemprop='streetAddress').get_text().strip()
    except:
        firma_str = None
    try:
        firma_plz = entry.find('span', itemprop='postalCode').get_text().strip()
    except:
        firma_plz = None
    try:
        firma_ort = entry.find('span', itemprop='addressLocality').get_text().strip()
    except:
        firma_ort = None
    try:
        firma_auth = entry.find('div', class_='authorization').get_text().strip()
    except:
        firma_auth = None
    try:
        firma_cert = entry.find('div', class_='cert-txt').get_text().strip()
    except:
        firma_cert = None
    try:
        firma_tel = entry.find('div', class_='icon-phone').find('a', itemprop="telephone").get_text().strip()
    except:
        firma_tel = None
    try:
        firma_mobile = entry.find('div', class_='icon-mobile').find('a', itemprop="telephone").get_text().strip()
    except:
        firma_mobile = None
    try:
        firma_email = entry.find('a', itemprop="email").get_text().strip()
    except:
        firma_email = None
    try:    
        firma_url = entry.find('a', itemprop="url").get_text().strip()
    except:
        firma_url = None
        
    return {'firma_name' : firma_name,
            'firma_href' : firma_href,
            'firma_str' : firma_str,
            'firma_plz' : firma_plz,
            'firma_ort' : firma_ort,
            'firma_auth' : firma_auth,
            'firma_tel' : firma_tel,
            'firma_mobile' : firma_mobile,
            'firma_email' : firma_email,
            'firma_url' : firma_url,
            'firma_cert' : firma_cert 
           }    

# Downloading data

Basis ist das WKÖ Firmenabc, hier für Standort Mödling (Gemeinde). Da eine Abfrage nur nach Ort nicht möglich ist, werden die einzelnen Branchen - auf höchster Ebene - durchgegangen.

In [ ]:
URLS = [
    'https://firmen.wko.at/Web/Ergebnis.aspx?StandortID=535&StandortName=M%C3%B6dling&Branche=3785&BranchenName=Gewerbe+und+Handwerk&CategoryID=0',
    'https://firmen.wko.at/Web/Ergebnis.aspx?StandortID=535&StandortName=M%C3%B6dling&Branche=3911&BranchenName=Bank+und+Versicherung&CategoryID=0',
    'https://firmen.wko.at/Web/Ergebnis.aspx?StandortID=535&StandortName=M%C3%B6dling&Branche=3876&BranchenName=Handel&CategoryID=0',
    'https://firmen.wko.at/Web/Ergebnis.aspx?StandortID=535&StandortName=M%C3%B6dling&Branche=3852&BranchenName=Industrie&CategoryID=0',
    'https://firmen.wko.at/Web/Ergebnis.aspx?StandortID=535&StandortName=M%C3%B6dling&Branche=3841&BranchenName=Information+und+Consulting&CategoryID=0',
    'https://firmen.wko.at/Web/Ergebnis.aspx?StandortID=535&StandortName=M%C3%B6dling&Branche=3935&BranchenName=Tourismus+und+Freizeitwirtschaft&CategoryID=0',
    'https://firmen.wko.at/Web/Ergebnis.aspx?StandortID=535&StandortName=M%C3%B6dling&Branche=3922&BranchenName=Transport+und+Verkehr&CategoryID=0',
    ]

In [ ]:
# Hinzufügen Parameter für Blättern durch Seiten
URLS = [url + '&Page={page}' for url in URLS]
# Falls nur Einträge mit "Kontaktdaten" (=mehr als nur Adresse, z.B. Tel, E-Mail, Homepage) abgefragt werden sollen
# kann man noch einen zusätzlichen Filter setzen
#URLS = [url + '&Filter=1&Page={page}' for url in URLS]

In [ ]:
%%time

df = pd.DataFrame()

for url in URLS:

    print(urllib.parse.parse_qs(url)['BranchenName'][0])

    page = 1
    while True:
        r = requests.get(url.format(page=page))
    #    if r.status_code != requests.codes.ok:
    #        break

        soup = BeautifulSoup(r.content, 'html.parser')

        if page == 1:
            print(soup.find('a', class_="treffer scroll").get_text())

        entries = soup.find_all('div', itemtype='http://schema.org/Organization')
        if len(entries) == 0:
            break

        for entry in entries:
            df = df.append(pd.DataFrame(extract_info(entry), index=[0]))

        print('Seite {} gelesen, letzter Eintrag {}'.format(page, extract_info(entry)['firma_name']))
        page += 1
        
df.to_pickle('df.pkl')
df.to_excel('firmen.xlsx', index=False)

In [ ]:
df = df.drop_duplicates(subset=['firma_href'])

In [ ]:
df.info()

In [ ]:
df['firma_branche'] = df.firma_href.map(lambda x:urllib.parse.parse_qs(x)['BranchenName'][0])

df['firma_id'] = df.firma_href.map(lambda x:urllib.parse.parse_qs(x)['/Web/DetailsKontakt.aspx?FirmaID'][0])

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.firma_id.nunique()

In [ ]:
df.firma_name.nunique()
# gives same result as len(set(df.firma_name))

In [ ]:
df.firma_plz.nunique()

In [ ]:
df.to_excel('firmen_wko.xlsx', index=False)

## Alternative source - not used, just as reference

Basis ist das Firmenabc für Mödling, hinten steht die Nummer der Seite: https://www.firmenabc.at/firmen/moedling_Llx/2

In [ ]:
URL = "https://www.firmenabc.at/firmen/moedling_Llx/{nr}"

In [ ]:
URL.format(nr=2)